# Introduction
Minimal intro to use the data cubes and the underlying hydrodynamical simulation (via scida).


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import h5py

In [ ]:
# for demonstration, use the low-res run of TNG50
# path to TNG simulation snapshot, make sure the groups_033 is in the same parent folder
path_snapshot = pathlib.Path("~/data/LyaLab/TNG50-4_z2/snapdir_033").expanduser()
# the Lya cube
path_lyacube = pathlib.Path("~/data/LyaLab/LyaCube_z2_v1d71da608.hdf5").expanduser()

## Lya data
### Lya emission
We provide uniform grids of the emission field as hdf5 files with the following structure. For example:

> ```text
> (bash) h5glance LyaCube_z2_v1d71da608.hdf5 --attrs
> LyaCube_z2_v1d71da608.hdf5
> ├6 attributes:
> │ ├author: 'Chris Byrohl <chris.byrohl@uni-heidelberg.de>'
> │ ├depth_angstrom: 42.69206657037045
> │ ├publication: 'Byrohl & Nelson 23,....org/abs/2212.08666'
> │ ├simulation series: 'TNG50_z2_rev2_fesc_v2'
> │ ├units: 'luminosity density in erg/s/pMpc^3'
> │ └width_arcsec: 2005.002916504041
> └maps
>   ├TNG50_total	[float64: 21 × 4010 × 4010]
>   ├TNG50_z2_rev2_SFR_fesc_v2	[float64: 21 × 4010 × 4010]
>   ├TNG50_z2_rev2_exc	[float64: 21 × 4010 × 4010]
>   └TNG50_z2_rev2_rec	[float64: 21 × 4010 × 4010]
> ```

The line of sight direction for photon propagation is along the x-axis. Note that this implies the observer is looking into the -x direction.

Given above metadata, we have a spectral resolution of roughly 2 Angstrom (rest-frame) and an angular resolution of roughly 0.5 arcseconds per element.

The emission fields are given in the "maps" group. Commonly, we give separate fields for the respective emission channel (after radiative transfer). In above example, we have the luminosity contributions from star-formation, diffuse recombinations and diffuse collisions (and their total).

#### Simple visualization
Plotting a *luminosity density* slice with 2 Angstrom depth.

In [ ]:
with h5py.File(path_lyacube, "r") as f:
    shape = f["maps/TNG50_total"].shape
    # slice at the center
    slc = f["maps/TNG50_total"][shape[0]//2][:]
    width_arcsec = f.attrs["width_arcsec"]

extent = [-width_arcsec/2, width_arcsec/2, -width_arcsec/2, width_arcsec/2]
pmin = np.percentile(slc, 1)
pmax = np.percentile(slc, 99.9)
plt.imshow(slc.T, origin="lower", norm="log", vmin=pmin, vmax=pmax, extent=extent)
plt.xlabel("arcsec")
plt.ylabel("arcsec")
plt.colorbar(label=r"luminosity density [erg/s/pMpc$^3$]")
plt.show()

#### Plotting a *surface brightness* slice with 6 Angstrom depth

In [ ]:
with h5py.File(path_lyacube, "r") as f:
    shape = f["maps/TNG50_total"].shape
    # slice at the center
    idx = shape[0]//2
    slc = f["maps/TNG50_total"][idx:idx+3, :, :].sum(axis=0)
    width_arcsec = f.attrs["width_arcsec"]

In [ ]:
from astropy.cosmology import Planck15
from lyalab.units import lumdens_to_sb
factor = lumdens_to_sb(shape).value
slc_sb = slc * factor

pmin = 1e-24
pmax = 1e-18
plt.imshow(slc_sb.T, origin="lower", norm="log", vmin=pmin, vmax=pmax, extent=extent)
plt.xlabel("arcsec")
plt.ylabel("arcsec")
plt.colorbar(label=r"luminosity density [arcsec$^2$ erg/s/cm$^2$]")
plt.show()



## Load complementary data via scida
We can conveniently access the underlying hydrodynamical simulation data, such as gas and galaxy properties, via [scida](https://scida.io).
In the following, we overplot galaxies with stellar masses above $\sim 10^{9.5} M_\odot$ within the slice.

In [ ]:
from scida import load
ds = load(path_snapshot)
halos = ds.data["Group"]
gals = ds.data["Subhalo"]
gas = ds.data["PartType0"]

smass = gals["SubhaloMassInRadType"][:,4].to("Msun").compute()
smass = smass.magnitude  # remove attached Msun unit
pos = gals["SubhaloPos"].compute().magnitude / ds.header["BoxSize"]
mask = pos[:,0] > (1.0 / shape[0]) * idx
mask &= pos[:,0] < (1.0 / shape[0]) * (idx+3)  # depth from last slice
mask &= smass > 3e9

pos = pos[mask] * width_arcsec - width_arcsec/2
smass = smass[mask]

plt.imshow(slc_sb.T, origin="lower", norm="log", vmin=pmin, vmax=pmax, extent=extent)
plt.colorbar(label=r"surface brightness [erg/s/cm$^2$/arcsec$^2$]")
plt.scatter(pos[:,1], pos[:,2], c=np.log10(smass), s=10, cmap="Reds")
plt.xlabel("arcsec")
plt.ylabel("arcsec")
plt.show()
